In [1]:
import os 
%pwd

'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPS Project Chicken Disease Classifier\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPS Project Chicken Disease Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict 
    params_image_size : list
    params_batch_size : list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:

        return EvaluationConfig(
            path_of_model = "artifacts/training/model.keras",
            training_data = "artifacts/data_ingestion/Chicken-Disease-Dataset",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

In [13]:
from urllib.parse import urlparse
import tensorflow as tf 

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        # Store evaluation config (model path, image size, batch size, data path, etc.)
        self.config = config
    
    def _valid_generator(self):
        # Normalize pixel values [0,1] and reserve 30% of data for validation
        datagenerator_kwargs = dict(rescale=1./255, validation_split=0.30)

        # Resize images to model's expected input (drops channel dim) and set batch size
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # e.g. (224,224,3) -> (224,224)
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"  # smooth resizing via bilinear interpolation
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Flow images from directory using only the validation subset; shuffle=False for consistent eval
        self.valid_generator = valid_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,  # important: keep order fixed during evaluation
            **dataflow_kwargs
        )

    @staticmethod 
    def load_model(path: Path) -> tf.keras.Model:
        # Load saved Keras model (supports SavedModel and .keras formats)
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        # Persist loss and accuracy to scores.json for experiment tracking
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e 

[2026-02-17 14:50:24,739: INFO: common: YAML file loaded successfully: config\config.yaml:]
[2026-02-17 14:50:24,745: INFO: common: YAML file loaded successfully: params.yaml:]
[2026-02-17 14:50:24,750: INFO: common: Directory created at: artifacts:]
Found 238 images belonging to 4 classes.
15/15 ━━━━━━━━━━━━━━━━━━━━ 102s 7s/step - accuracy: 0.4832 - loss: 8.3222
[2026-02-17 14:52:50,543: INFO: common: JSON file saved at: scores.json:]
